In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv('COHERE_API_KEY', '')

In [2]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
import json
import cohere

c:\Users\heena\Desktop\Genetative_AI\genai_env\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [7]:
co = cohere.Client(api_key=cohere_api_key)

### Step 1. Create the Tool
- A tool can be any function or service that can receive and send data. 
- It could be an email service, an SQL database, a vector database, a weather data service, a sports data service, a web search engine, or even another LLM.
- Here we will use the example of a dummy database containing only 3 entries

In [9]:
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}
    

functions_map = {
    "daily_sales_report": daily_sales_report
}

### Step 2 - Define Tool Schema
- After defining the tool, next we define the tool schema, which is a description of what the tool does and what are the parameters it takes.
- The schema must contain the following fields:
    - `name`: the name of the tool.
    - `description`: a description of what the tool is and what it is used for.
    - `parameter_definitions`: a list of parameters that the tool accepts. For each parameter, we need to define the following fields:
        - `description` details about the parameter
        - `type` the parameter’s data type, e.g., str, int, and so on
        - `required` (either True or False) fields
- This schema informs the LLM about what the tool does, and the LLM decides whether to use a particular tool based on it.
- Therefore, the more descriptive and specific the schema, the more likely the LLM will make right tool call decisions.


In [10]:
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    }
]

### Step 3 Create Custom Preamble (Optional)
- It’s a completely optional step, though it’s likely needed if we want to create a robust and reliable application. 
- Also note that the preamble is not related to the tool setup, rather it’s part of the instruction to the LLM.
- **Preamble Guide**: https://docs.cohere.com/docs/preambles?ref=cohere-ai.ghost.io

In [11]:
preamble = """## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

### Step 4 Running the tool workflow
- The tool workflow mainly consists for following four components:
    - The user
    - The application
    - The LLM
    - The tools

- At its most basic, these four components interact in a workflow through four steps:
    - Step 1: Get user message. The LLM gets the user message (via the application).
    - Step 2: Generate tool calls. The LLM decides which tools to call (if any) and generates the tool calls.
    - Step 3: Get tool results. The application executes the tools, and the results are sent to the LLM.
    - Step 4: Generate response and citations. The LLM generates the response and citations back to the user.

In [12]:
# 1. The application receives the user message
message = "Can you provide a sales summary for 29th September 2023?"

In [13]:
model = "command-r-plus"

# Initial response to the user message
response = co.chat(
    message=message,
    model=model,
    preamble=preamble,
    tools=tools,
    force_single_step=True
)
tool_calls = response.tool_calls

In [14]:
tool_calls

[ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'})]

In [15]:
print("Tool calls:\n")
for i, t in enumerate(tool_calls):
    print(f"#{i+1}\nTool: {t.name}\nParameters: {t.parameters}\n")

Tool calls:

#1
Tool: daily_sales_report
Parameters: {'day': '2023-09-29'}



In [19]:
chat_history = response.chat_history

In [16]:
# Now call the function for the tool selected by the LLM
tool_results = []
for tc in tool_calls:
    tool_output = functions_map[tc.name](**tc.parameters)
    tool_call = {"name": tc.name, "parameters": tc.parameters}
    tool_results.append({"call": tool_call, "outputs": [tool_output]})

In [17]:
tool_results

[{'call': {'name': 'daily_sales_report', 'parameters': {'day': '2023-09-29'}},
  'outputs': [{'date': '2023-09-29',
    'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}]}]

In [21]:
# Send the generated results to the LLM and generate response for the user
# Generate response
final_response = co.chat(
    message="",
    model=model,
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)
    
print("Final response:")
print(final_response.text)
print("="*50)

Final response:
On 29 September 2023, the total sales amount was 10,000 and the total units sold was 250.


In [22]:
# get the citations
if final_response.citations:
    print("\nCitations:")
    for citation in final_response.citations:
        print(citation)
    print("\nCited Documents:")
    for document in final_response.documents:
        print(document)


Citations:
start=26 end=55 text='total sales amount was 10,000' document_ids=['daily_sales_report:0:2:0']
start=64 end=88 text='total units sold was 250' document_ids=['daily_sales_report:0:2:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}


### Tool Use Modes
- A good tool use API must be able to handle different scenarios, from simple to complex LLM tool calls. 
- This translates to being able to call tools in different modes: single-step, multi-step, and parallel.
- For details Refer - https://cohere.com/llmu/tool-use-anatomy?ref=cohere-ai.ghost.io